# Case Study Model selection for Clustering
 

Clustering is unsupervised learning: the resulting clusters are completely derived from data distributed in given a feature set with no class available

Compared to supervised learning counterparts, it is …
* hard to define model performance (cluster quality)
* sensitive to different clustering algorithms and different feature spaces.



#### Task
Your task is to try different clustering algorithms and also a range of the potential parameter(s) which affect the number of clusters including ..

* K-means
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
* Gaussian Mixture Model, 
https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html#sklearn.mixture.GaussianMixture
* Hierarchical Clustering, 
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering
* Louvain Clustering, 
https://scikit-network.readthedocs.io/en/latest/reference/clustering.html#module-sknetwork.clustering

on 5K colorectal patches represented by 4 different representation PathologyGAN, ResNet50, InceptionV3 and VGG16


#### Data and its preprocessing 
5,000 non-overlapping image patches from hematoxylin & eosin (H&E) stained histological images of human colorectal cancer (CRC) and normal tissue.
* 4 feature sets, PathologyGAN, ResNet50, InceptionV3 and VGG16, are extracted to represent those 5,000 images different dimensional feature spaces.
* PCA and UMAP were employed to reduce each feature sapce into 100-dimensional vectors

* 9 tissue types are also available which include Adipose (ADI), background (BACK), debris (DEB), lymphocytes (LYM), mucus (MUC), smooth muscle (MUS), normal colon mucosa (NORM), cancer-associated stroma (STR), colorectal adenocarcinoma epithelium (TUM)


#### Performance Measurement
To assess quality of clustering solutions, several approaches are expected to be done and interpreted which include...
* Silhouette Score for goodness of fit test
* Vmeasure Score for homogeneity and completeness test (tissue type available as ground truth)
* Clusters visualisations

For more information, please have a check...
https://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation


#### Report
Report on your preprocessing pipeline, theory and intuition behinds each algorithm and representation, parameter searching and performance evaluation frameworks. If there is any addiotional process, give evidences/justifications on how it helps.

#### Required Packages

In [1]:
# !pip install h5py==2.10.0
# !pip install numpy
# !pip install pandas
# !pip install sklearn
# !pip install scikit-network
# !pip install pickle-mixin==1.0.2
# !pip install matplotlib
# !pip install plotly

In [1]:
# Data manipulation and computation
import numpy as np
import pandas as pd
import h5py
import random

# Machine Learning and Data Preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score, pairwise_distances

# Clustering
from sknetwork.clustering import Louvain
from scipy import sparse

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Others
import pickle

## Data Loading and Preprocessing

In [2]:
pge_path = 'colon_nct_feature/pge_dim_reduced_feature.h5'
resnet50_path = 'colon_nct_feature/resnet50_dim_reduced_feature.h5'
inceptionv3_path = 'colon_nct_feature/inceptionv3_dim_reduced_feature.h5'
vgg16_path = 'colon_nct_feature/vgg16_dim_reduced_feature.h5'




In [6]:
pge_content = h5py.File(pge_path, mode='r')
resnet50_content = h5py.File(resnet50_path, mode='r')
inceptionv3_content = h5py.File(inceptionv3_path, mode='r')
vgg16_content = h5py.File(vgg16_path, mode='r')

In [ ]:
scaler = MinMaxScaler()
pge_normalized = scaler.fit_transform(pge_content)
resnet50_normalized = scaler.fit_transform(resnet50_content)
inceptionv3_normalized = scaler.fit_transform(inceptionv3_content)
vgg16_normalized = scaler.fit_transform(vgg16_content)

In [ ]:
datasets = {
    'pge': pge_normalized,
    'resnet50': resnet50_normalized,
    'inceptionv3': inceptionv3_normalized,
    'vgg16': vgg16_normalized
}

In [3]:
# Load the numerical data (e.g., 'pca_feature') from the h5 files
def load_data(path, key):
    with h5py.File(path, 'r') as h5_file:
        data = h5_file[key][...]
        return data

# Use the 'pca_feature' or 'umap_feature' key as needed
pge_data = load_data(pge_path, 'pca_feature')
resnet50_data = load_data(resnet50_path, 'pca_feature')
inceptionv3_data = load_data(inceptionv3_path, 'pca_feature')
vgg16_data = load_data(vgg16_path, 'pca_feature')

# Normalize the data
scaler = MinMaxScaler()
pge_normalized = scaler.fit_transform(pge_data)
resnet50_normalized = scaler.fit_transform(resnet50_data)
inceptionv3_normalized = scaler.fit_transform(inceptionv3_data)
vgg16_normalized = scaler.fit_transform(vgg16_data)

In [4]:
def load_and_normalize_data(path, key):
    with h5py.File(path, 'r') as h5_file:
        data = h5_file[key][...]
        return MinMaxScaler().fit_transform(data)

datasets = {
    'pge_pca': load_and_normalize_data(pge_path, 'pca_feature'),
    'pge_umap': load_and_normalize_data(pge_path, 'umap_feature'),
    'vgg16_pca': load_and_normalize_data(vgg16_path, 'pca_feature'),
    'vgg16_umap': load_and_normalize_data(vgg16_path, 'umap_feature'),
}

In [7]:
def extract_labels(h5_content):
    filename = np.squeeze(h5_content['file_name'])
    filename = np.array([str(x) for x in filename])
    labels = np.array([x.split('/')[2] for x in filename])
    return labels

# Extracting labels for each dataset
pge_labels = extract_labels(pge_content)
resnet50_labels = extract_labels(resnet50_content)
inceptionv3_labels = extract_labels(inceptionv3_content)
vgg16_labels = extract_labels(vgg16_content)

# You may store these labels in a dictionary for easy access
labels_dict = {
    'pge': pge_labels,
    'resnet50': resnet50_labels,
    'inceptionv3': inceptionv3_labels,
    'vgg16': vgg16_labels
}


In [ ]:
#PCA feature from 4 feature sets: pge_latent, resnet50_latent, inceptionv3_latent, vgg16_latent
pge_pca_feature  = pge_content['pca_feature'][...]
resnet50_pca_feature  = resnet50_content['pca_feature'][...]
inceptionv3_pca_feature = inceptionv3_content['pca_feature'][...]
vgg16_pca_feature  = vgg16_content['pca_feature'][...]

In [ ]:
#UMAP feature from 4 feature sets: pge_latent, resnet50_latent, inceptionv3_latent, vgg16_latent
pge_umap_feature  = pge_content['umap_feature'][...]
resnet50_umap_feature = resnet50_content['umap_feature'][...]
inceptionv3_umap_feature  = inceptionv3_content['umap_feature'][...]
vgg16_umap_feature  = vgg16_content['umap_feature'][...]

In [8]:
#tissue type as available ground-truth: labels
filename  = np.squeeze(pge_content['file_name'])
filename = np.array([str(x) for x in filename])
labels = np.array([x.split('/')[2] for x in filename])
labels

NameError: name 'pge_content' is not defined

In [8]:
# Parameters for Louvain clustering
resolutions = [0.9, 1, 0.8]
modularity_options = ['Dugue', 'Newman', 'Potts']
random_state = 0

# Function to transform distances into similarities
def create_similarity_matrix(data):
    distances = pairwise_distances(data)
    # Convert distances to similarities, for example using an exponential function
    # You can adjust this transformation as needed
    max_distance = np.max(distances)
    similarities = np.exp(-(distances ** 2) / (2. * max_distance ** 2))
    return sparse.csr_matrix(similarities)

In [11]:
from sklearn.metrics import v_measure_score

louvain_results = []

for dataset_name, data in datasets.items():
    true_labels = labels_dict[dataset_name.split('_')[0]]  # Extracting the corresponding true labels
    similarity_matrix = create_similarity_matrix(data)

    for resolution in resolutions:
        for modularity in modularity_options:
            louvain_model = Louvain(resolution=resolution, modularity=modularity, random_state=random_state)
            labels = louvain_model.fit_predict(similarity_matrix)

            # Compute silhouette score
            if len(np.unique(labels)) > 1:
                silhouette = silhouette_score(data, labels)
            else:
                silhouette = None  # or a placeholder value, e.g., -1
                print(f"Invalid number of unique labels: {len(np.unique(labels))} for dataset {dataset_name}")

            # Compute V-measure score
            v_measure = v_measure_score(true_labels, labels)

            # Store results
            louvain_results.append({
                'dataset': dataset_name,
                'resolution': resolution,
                'modularity': modularity,
                'silhouette_score': silhouette,
                'v_measure_score': v_measure,
                'cluster_size': len(np.unique(labels))
            })

# Convert results to DataFrame for easier analysis
results_df = pd.DataFrame(louvain_results)


Invalid number of unique labels: 1 for dataset pge_pca
Invalid number of unique labels: 1 for dataset pge_pca
Invalid number of unique labels: 1 for dataset pge_pca
Invalid number of unique labels: 1 for dataset pge_pca
Invalid number of unique labels: 1 for dataset pge_pca
Invalid number of unique labels: 1 for dataset pge_umap
Invalid number of unique labels: 1 for dataset pge_umap
Invalid number of unique labels: 1 for dataset pge_umap
Invalid number of unique labels: 1 for dataset pge_umap
Invalid number of unique labels: 1 for dataset vgg16_pca
Invalid number of unique labels: 1 for dataset vgg16_pca


ValueError: Number of labels is 5000. Valid values are 2 to n_samples - 1 (inclusive)

In [14]:
# Smaller increments in the range 1 to 1.5
resolution_values = [1.0, 1.1, 1.2, 1.3, 1.4]

for resolution in resolution_values:
    similarity_matrix = create_similarity_matrix(data_to_test)
    louvain_model = Louvain(resolution=resolution, random_state=0)
    labels = louvain_model.fit_predict(similarity_matrix)

    num_clusters = len(np.unique(labels))
    print(f"Resolution: {resolution}, Number of clusters: {num_clusters}")

    # Check if silhouette score is applicable
    if 1 < num_clusters < len(data_to_test):
        silhouette = silhouette_score(data_to_test, labels)
        print(f"Silhouette Score for resolution {resolution}: {silhouette}")
    else:
        print(f"Silhouette score not applicable for resolution {resolution}")


Resolution: 1.0, Number of clusters: 3
Silhouette Score for resolution 1.0: 0.05235109105706215
Resolution: 1.1, Number of clusters: 5000
Silhouette score not applicable for resolution 1.1
Resolution: 1.2, Number of clusters: 5000
Silhouette score not applicable for resolution 1.2
Resolution: 1.3, Number of clusters: 5000
Silhouette score not applicable for resolution 1.3
Resolution: 1.4, Number of clusters: 5000
Silhouette score not applicable for resolution 1.4


In [15]:
# Example for pge_data with resolution = 1.0
similarity_matrix = create_similarity_matrix(pge_data)
louvain_model = Louvain(resolution=1.0, random_state=0)
pge_labels = louvain_model.fit_predict(similarity_matrix)

# Check the number of clusters and calculate silhouette score if applicable
num_clusters = len(np.unique(pge_labels))
print(f"Number of clusters for pge_data: {num_clusters}")

if 1 < num_clusters < len(pge_data):
    silhouette = silhouette_score(pge_data, pge_labels)
    print(f"Silhouette Score for pge_data: {silhouette}")
else:
    print("Silhouette score not applicable for pge_data")

# Repeat similar steps for other datasets with their optimal resolutions


Number of clusters for pge_data: 3
Silhouette Score for pge_data: 0.05235109105706215


In [13]:
from sknetwork.clustering import Louvain
from scipy import sparse

def create_similarity_matrix(data):
    distances = pairwise_distances(data)
    max_distance = np.max(distances)
    similarities = np.exp(-(distances ** 2) / (2. * max_distance ** 2))
    return sparse.csr_matrix(similarities)

# Select a dataset to experiment with
data_to_test = pge_data  # Replace with other datasets as needed

# Range of resolution values to test
resolution_values = [0.1, 0.5, 1, 1.5, 2]  # Adjust this range based on results

for resolution in resolution_values:
    similarity_matrix = create_similarity_matrix(data_to_test)
    louvain_model = Louvain(resolution=resolution, random_state=0)
    labels = louvain_model.fit_predict(similarity_matrix)

    num_clusters = len(np.unique(labels))
    print(f"Resolution: {resolution}, Number of clusters: {num_clusters}")

    # Optional: if you get more than 1 cluster, check the silhouette score
    if num_clusters > 1:
        silhouette = silhouette_score(data_to_test, labels)
        print(f"Silhouette Score for resolution {resolution}: {silhouette}")


Resolution: 0.1, Number of clusters: 1
Resolution: 0.5, Number of clusters: 1
Resolution: 1, Number of clusters: 3
Silhouette Score for resolution 1: 0.05235109105706215
Resolution: 1.5, Number of clusters: 5000


ValueError: Number of labels is 5000. Valid values are 2 to n_samples - 1 (inclusive)

## Example

In [ ]:
import random

In [ ]:
random.seed(0)
selected_index = random.sample(list(np.arange(len(pge_pca_feature))), 200)

In [ ]:
test_data = pge_pca_feature[selected_index]
test_label = labels[selected_index]

### Exploratory Analysis

In [ ]:
import plotly.graph_objects as go
import pandas as pd

In [ ]:
traces = []
for name in np.unique(labels):
    trace = go.Scatter3d(
        x=test_data[test_label==name,0],
        y=test_data[test_label==name,1],
        z=test_data[test_label==name,2],
        mode='markers',
        name=name,
        marker=go.scatter3d.Marker(
            size=4,
            opacity=0.8
        )

    )
    traces.append(trace)


data = go.Data(traces)
layout = go.Layout(
            showlegend=True,
    scene=go.Scene(
                xaxis=go.layout.scene.XAxis(title='PC1'),
                yaxis=go.layout.scene.YAxis(title='PC2'),
                zaxis=go.layout.scene.ZAxis(title='PC3')
                )
)
fig = go.Figure(data=data, layout=layout)
fig.update_layout(
    title="First 3 pricipal components of PathologyGAN's PCA feature",
    legend_title="Legend Title",
)

fig.show()


## Model training

In [ ]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering
from sknetwork.clustering import Louvain


In [ ]:
#to create Adjacency matrix for  Louvain clustering
from sklearn.metrics import pairwise_distances 
from sklearn.preprocessing import MinMaxScaler


In [ ]:
kmeans_model = KMeans(n_clusters = 3, random_state = 0) #GaussianMixture(), AgglomerativeClustering(), Louvain
kmeans_assignment = kmeans_model.fit_predict(test_data)

In [ ]:
from scipy import sparse
louvain_model = Louvain(resolution = 0.9, modularity = 'Newman',random_state = 0) 
adjacency_matrix = sparse.csr_matrix(MinMaxScaler().fit_transform(-pairwise_distances(test_data)))
louvain_assignment = louvain_model.fit_transform(adjacency_matrix)

### Evaluation and Visualisation

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score, v_measure_score
from sklearn.model_selection import KFold, train_test_split

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
# Normalize the PCA features for each dataset
datasets = {
    'pge': pge_pca_feature,
    'resnet50': resnet50_pca_feature,
    'inceptionv3': inceptionv3_pca_feature,
    'vgg16': vgg16_pca_feature
}

normalized_datasets = {key: MinMaxScaler().fit_transform(value) for key, value in datasets.items()}

# Parameters for Louvain clustering
resolutions = [0.9, 1, 0.8]
modularity_options = ['Dugue', 'Newman', 'Potts']
random_state = 0

# Initialize results storage
louvain_results = []

# Apply Louvain Clustering to each dataset
for dataset_name, data in normalized_datasets.items():
    for resolution in resolutions:
        for modularity in modularity_options:
            louvain_model = Louvain(resolution=resolution, modularity=modularity, random_state=random_state)

            # Transform distances into similarities
            distances = euclidean_distances(data)
            max_distance = np.max(distances)
            adjacency_matrix = sparse.csr_matrix(np.exp(-(distances ** 2) / (2. * max_distance ** 2)))

            # Evaluation Metrics (using Silhouette Score)
            silhouette = silhouette_score(data, labels)

            # Store results
            louvain_results.append({
                'dataset': dataset_name,
                'resolution': resolution,
                'modularity': modularity,
                'silhouette_score': silhouette,
                'cluster_size': len(np.unique(labels))
            })

# Convert results to DataFrame for analysis
results_df = pd.DataFrame(louvain_results)

# Convert results to DataFrame for easier analysis
results_df = pd.DataFrame(louvain_results)

# Plotting the results
plt.figure(figsize=(10, 6))
sns.lineplot(data=results_df, x='cluster_size', y='silhouette_score', hue='resolution', style='modularity', markers=True)
plt.title('Louvain Clustering Performance')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.legend(title='Resolution and Modularity')
plt.show()

# Summary of Best Configurations
best_configurations = results_df.sort_values(by='silhouette_score', ascending=False).groupby(['resolution', 'modularity']).first().reset_index()
print("Best Configurations by Resolution and Modularity:")
print(best_configurations[['resolution', 'modularity', 'cluster_size', 'silhouette_score']])



* check out number of clusters/cluster assignment counts

In [ ]:
print('Number of clusters from KMeans: %d and from Louvain: %d'%(np.unique(kmeans_assignment).shape[0],np.unique(louvain_assignment).shape[0]))

In [ ]:
kmeans_counts = np.unique(kmeans_assignment, return_counts = True)
louvain_counts = np.unique(louvain_assignment, return_counts = True)

In [ ]:
print('Kmeans assignment counts')
pd.DataFrame({'Cluster Index': kmeans_counts[0], 'Number of members':kmeans_counts[1]}).set_index('Cluster Index')

In [ ]:
print('Louvain assignment counts')
pd.DataFrame({'Cluster Index': louvain_counts[0], 'Number of members':louvain_counts[1]}).set_index('Cluster Index')

* Assess goodness of fit by silhouette score and cluster homogeneities by V-measure

In [ ]:
kmeans_silhouette = silhouette_score(test_data, kmeans_assignment)
louvain_silhouette = silhouette_score(test_data, louvain_assignment)
kmeans_v_measure = v_measure_score(test_label, kmeans_assignment)
louvain_v_measure = v_measure_score(test_label, louvain_assignment)
pd.DataFrame({'Metrics': ['silhouette', 'V-measure'], 'Kmeans': [kmeans_silhouette, kmeans_v_measure], 'Louvain':[louvain_silhouette, louvain_v_measure]}).set_index('Metrics')

* Visualise tissue type percentage in two different clustering configurations

In [ ]:
def calculate_percent(sub_df, attrib):
    cnt = sub_df[attrib].count()
    output_sub_df = sub_df.groupby(attrib).count()
    return (output_sub_df/cnt)

In [ ]:
resulted_cluster_df = pd.DataFrame({'clusterID': kmeans_assignment, 'type': test_label})
label_proportion_df = resulted_cluster_df.groupby(['clusterID']).apply(lambda x: calculate_percent(x,'type')).rename(columns={'clusterID':'type_occurrence_percentage'}).reset_index()
pivoted_label_proportion_df = pd.pivot_table(label_proportion_df, index = 'clusterID', columns = 'type', values = 'type_occurrence_percentage')


f, axes = plt.subplots(1, 2, figsize=(20,5))
number_of_tile_df = resulted_cluster_df.groupby('clusterID')['type'].count().reset_index().rename(columns={'type':'number_of_tile'})
df_idx = pivoted_label_proportion_df.index
(pivoted_label_proportion_df*100).loc[df_idx].plot.bar(stacked=True, ax = axes[0] )

axes[0].set_ylabel('Percentage of tissue type')
axes[0].legend(loc='upper right')
axes[0].set_title('Cluster configuration by Kmeans')

resulted_cluster_df = pd.DataFrame({'clusterID': louvain_assignment, 'type': test_label})
label_proportion_df = resulted_cluster_df.groupby(['clusterID']).apply(lambda x: calculate_percent(x,'type')).rename(columns={'clusterID':'type_occurrence_percentage'}).reset_index()
pivoted_label_proportion_df = pd.pivot_table(label_proportion_df, index = 'clusterID', columns = 'type', values = 'type_occurrence_percentage')


number_of_tile_df = resulted_cluster_df.groupby('clusterID')['type'].count().reset_index().rename(columns={'type':'number_of_tile'})
df_idx = pivoted_label_proportion_df.index
(pivoted_label_proportion_df*100).loc[df_idx].plot.bar(stacked=True, ax = axes[1] )

axes[1].set_ylabel('Percentage of tissue type')
axes[1].legend(loc='upper right')
axes[1].set_title('Cluster configuration by Louvain')
f.show()